In [8]:
%%capture

import os
import sys
from os.path import join as pjoin
import pickle as pkl

from gensim.models.word2vec import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

sys.path.append('..')
from tst.preprocess.w2v_extensions import *
from tst.preprocess.corpus import CorpusStreamer
from tst.io import AUTHORS

'/mnt/c/Users/kevin/Documents/workspace/text-style-transfer/data/authors'

In [ ]:
# create and train
for author in os.listdir(AUTHORS):
    if author == 'all':
        continue
    stream = CorpusStreamer(pjoin(AUTHORS, author))
    w2v = Word2Vec.create_and_train(stream, epochs=20)
    w2v.wv.save_word2vec_format(pjoin(AUTHORS, author, 'parsed', 'only.vec'))

In [ ]:
# load and tune
for author in os.listdir(AUTHORS):
    if author == 'all':
        continue
    w2v = Word2Vec.load("./w2v_models/gutenberg_w2v_5e.model")
    stream = CorpusStreamer(f'../DataAcquisition/data/{author}/')
    w2v.build_vocab(stream, update=True)
    w2v.train(stream, total_examples=sum(1 for x in stream), epochs = 20)
    w2v.save(f'../DataAcquisition/data/{author}/parsed/tuned.model')

In [71]:
# shrink embeddings
for author in os.listdir(AUTHORS):
    print(author)
    author_path = pjoin(AUTHORS, author, 'parsed')
    Word2Vec.load(pjoin(author_path, 'tuned.model')).wv\
        .save_word2vec_format_reduced(pjoin(author_path, 'reduced.vec'),
                                     words=pjoin(author_path, 'dict.txt'),
                                     topn=2e5)

chesterton
darwin
dickens
verne
wells


In [11]:
vecs = {}
for author in os.listdir(AUTHORS):
    try:
        vecs[author] = KeyedVectors.load_word2vec_format(pjoin(AUTHORS, author, 'parsed', 'only.vec'))
    except:
        pass

In [45]:
for k,v in vecs.items():
    print(k, len(v.vocab), np.unique([w.lemma_ for w in nlp(' '.join(v.vocab.keys())) if w.is_alpha]).size)

chesterton 36362 23362
darwin 51073 32255
dickens 57299 32335
verne 48837 32598
wells 53757 32793


In [56]:
pjoin(AUTHORS, author)

'/mnt/c/Users/kevin/Documents/workspace/text-style-transfer/data/authors/churchill'

In [60]:
author = 'twain'
stream = CorpusStreamer(pjoin(AUTHORS, author, 'books'))
w2v = Word2Vec(size=100, window=5, min_count=1, workers=4)
w2v.build_vocab(stream)
w2v.train(stream, total_examples=w2v.corpus_count, epochs=20)
w2v.wv.save_word2vec_format(pjoin(AUTHORS, author, 'parsed', 'only.vec'))